# Sora2 Watermark Remover - Simple Version

シンプルな6ステップでSora動画からウォーターマークを除去します。

**リポジトリ:** https://github.com/fulfulggg/Sora2WatermarkRemover

## 1. 依存関係のインストール

In [ ]:
!pip install -q iopaint transformers

## 2. LaMaモデルのダウンロード

In [ ]:
!iopaint download --model lama

## 3. 最新コードの取得

In [ ]:
!curl -sL https://github.com/fulfulggg/Sora2WatermarkRemover/archive/refs/heads/main.tar.gz | tar xz --strip-components=1
print("\n✅ リポジトリのダウンロードが完了しました。")

## 4. 動画のアップロード

In [ ]:
from google.colab import files

print("動画ファイルをアップロードしてください:")
uploaded = files.upload()

input_file = list(uploaded.keys())[0]
print(f"\n✅ アップロード完了: {input_file}")

## 5. ウォーターマークの除去

パラメータ:
- `--max-bbox-percent`: バウンディングボックスの最大サイズ（デフォルト: 15%）
- `--force-format`: 出力形式（MP4/AVI）
- `--overwrite`: 既存ファイルの上書き

In [ ]:
!python remwm.py "{input_file}" "/content/out.mp4" \
  --max-bbox-percent=15 \
  --force-format=MP4 \
  --overwrite

## 6. 結果のダウンロード

In [ ]:
print("結果をダウンロード中...")
files.download('/content/out.mp4')
print("\n✅ 完了！")

---

## オプション1: 固定位置ウォーターマーク除去（delogo）

**推奨：** AI検出で一部フレームにウォーターマークが残る場合は、この方法を使用してください。

### 特徴
- ✅ **確実**: 固定位置のウォーターマークを100%除去
- ✅ **高速**: AI検出不要、FFmpegで直接処理
- ✅ **簡単**: 座標を指定するだけ

### ステップ1: 位置確認（プレビュー）

まず3秒分のプレビューでウォーターマークの位置を確認します。

In [ ]:
# プレビュー生成（枠表示で位置調整）
# x,y,w,h を調整してウォーターマークを完全に覆うようにする
!ffmpeg -y -i "{input_file}" -vf "delogo=x=80:y=80:w=320:h=120:show=1" -t 3 /content/preview.mp4

print("\n✅ プレビュー生成完了: /content/preview.mp4")
print("枠がウォーターマークを完全に覆うまで、x,y,w,hを調整してください")

### ステップ2: 本番処理（delogo）

位置が確認できたら、本番処理を実行します。

In [ ]:
# 本番処理（delogoで確実除去）
# ステップ1で確認した座標を使用
!python remwm.py "{input_file}" "/content/out.mp4" \
  --delogo "80,80,320,120" \
  --force-format=MP4 \
  --overwrite

print("\n✅ 処理完了！")

### 複数の位置にウォーターマークがある場合

複数の`--delogo`オプションを指定できます：

```python
!python remwm.py "{input_file}" "/content/out.mp4" \
  --delogo "80,80,320,120" \
  --delogo "1200,780,340,130" \
  --force-format=MP4 \
  --overwrite
```

---

## オプション2: AI検出のパラメータ調整

AI検出を使用する場合（ステップ5）、以下のパラメータで調整できます：

```python
!python remwm.py "{input_file}" "/content/out.mp4" \
  --max-bbox-percent=22 \
  --frame-step=1 \
  --target-fps=0 \
  --force-format=MP4 \
  --overwrite
```

- `--max-bbox-percent`: バウンディングボックスの最大サイズ（デフォルト: 10%）
- `--frame-step`: フレーム処理間隔（1=全フレーム、2=隔フレーム...）
- `--target-fps`: 出力fps指定（0=入力と同じ）